# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [22]:
import sys
sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
from typing import Callable
from datetime import datetime

In [6]:
original_write = sys.stdout.write

In [7]:
print('1, 2, 3')
print(original_write('1, 2, 3'))# печатает кол-во байт сразу

1, 2, 3
1, 2, 37


In [8]:
flg = True
def my_write(string_text: str) -> int:
    global flg
    time_text = f'[{datetime.now():%Y-%m-%d %H:%M}]: '
    if flg:
        string_text = f'{time_text}{string_text}'
        flg = False
    if string_text == '\n':
        flg = True
    else:
        string_text = string_text.replace('\n', f'\n{time_text}')
    return original_write(string_text)


sys.stdout.write = my_write

In [9]:
print('1, 2, 3')
print('1, 2, 3')
print(my_write('1, 2, 3'))
print('1', '2', sep = '\n')


[2022-12-17 15:52]: 1, 2, 3
[2022-12-17 15:52]: 1, 2, 3
[2022-12-17 15:52]: 1, 2, 327
[2022-12-17 15:52]: 1
[2022-12-17 15:52]: 2


In [10]:
print('1, 2, 3')

[2022-12-17 15:52]: 1, 2, 3


In [11]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [12]:
def timed_output(function: Callable) -> Callable:
    original_write = sys.stdout.write
    flg = True

    def my_write(string_text: str) -> int:
        global flg
        time_text = f'[{datetime.now():%Y-%m-%d %H:%M}]: '
        if flg:
            string_text = f'{time_text}{string_text}'
            flg = False
        if string_text == '\n':
            flg = True
        else:
            string_text = string_text.replace('\n', f'\n{time_text}')
        return original_write(string_text)



    def wrapper(name: str):
        sys.stdout.write = my_write
        result = function(name)
        sys.stdout.write = original_write
        return result

    return wrapper

In [13]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [14]:
print_greeting("Nikita")

[2022-12-17 15:52]: Hello, Nikita!


In [15]:
print('1, 2')

1, 2


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [16]:
import functools


In [17]:

def redirect_output(filepath: str, *args, **kwargs) -> Callable:
    def outer_wrapper(function: Callable) -> Callable:
        def inner_wrapper(*args, **kwargs):
            original_stdout = sys.stdout
            with open(filepath, 'w', *args, **kwargs) as file_to_write_output:
                sys.stdout = file_to_write_output
                result = function(*args, **kwargs)
            sys.stdout = original_stdout
            return result
        return inner_wrapper
    return outer_wrapper

In [18]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ') 
        print()

In [19]:
calculate()

In [20]:
print(10)

10


In [21]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
